# Build a decision tree

In [107]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import collections
%matplotlib inline

In [581]:
# ID3/ C4.5/ CART 

class DecisionTreeClassifier:
    """
    Decision Tree Classifier.
    
    Parameters
    ------------------
    verbose: int {1,0}
        0. Print nothing
        1. Print each steps
        
    criterion：string
        1. Gini: gini impurity
        2. InfoGain: information gain
        3. GainRatio: information gain ratio
    
    max_depth: int
        The maximum depth of the tree. 
        If None, then nodes are expanded until all leaves are pure 
        or until all leaves contain less than min_samples_split samples.
    
    min_impurity_decrease:
    
    min_samples_leaf
    
    max_leaf_nodes
    
    min_impurity_split
    
    
        
    Attributes
    ------------------
    
    """
    
    
    def __init__(self,criterion = 'InfoGain', verbose = 1):
        self.criterion = criterion
        if self.criterion == 'InfoGain':
            self.feature_split = self._feature_Select_Info_Gain
        elif self.criterion == 'GainRatio':
            self.feature_split = self._feature_Select_Info_Gain_Ratio
        elif self.criterion == 'Gini':
            self.feature_split = self._feature_Select_Gini
        self.verbose = verbose
        
    """
    Step1: FEATURE SELECTION
    
    Calculate Entropy or Gini index 

    """
    # Calculate entropy
    
    def calcShannonEntropy(self,y_labels):
        """
        Parameters
        ------------------
        y_labels: {array-like}, shape = [n_samples, 1]
            Dateset matrix, where 'n_samples' is the number of samples 
            and one column of labels
        
        Return
        ------------------
        shannonEntropy: float
        
        """
        m = len(y_labels)
        
        labelCounts = collections.defaultdict(int)
        
        # the number of unique elements and their occurrence
        for label in y_labels:

            if label not in labelCounts:
                labelCounts[label] = 0
            labelCounts[label] += 1
            
        # calculate shannon entropy
        shannonEntropy = 0.0
        for key in labelCounts:
            # calculate occurrence
            prob = float(labelCounts[key]/m)
            if prob == 0:
                continue
            # calculate entropy
            shannonEntropy -= prob * np.log2(prob)
        
        return shannonEntropy
    
    # calculate conditional entropy
    def conditionalEnropy(self, X, y_label, feature, split_point):#, split_point
        """
        Parameters
        ------------------
        X: {array-like}
        preprocessed dataset, could not deal with continuous features, 
        and categorical feature should be better as binary form
        
        y_label: {array-like}
        
        feature: string
        
        split_point: int/float/string (can only be discrete)
        
        """
        m,n = X.shape
        
        X_cond = X[:,feature]

        for name in range(m):
            if X_cond[name] != split_point:
                X_cond[name] = 1
            else:
                X_cond[name] = 0
        # Create a nested dictionary, 
        # {"X_cate":{"label":count}}
        
        # Find unique category in X_cond
        X_cate = set(X_cond)

        
        # Find unique label in y_label
        label = set(y_label)
        
        
        
        labelCounts = collections.defaultdict(dict)
        
        for cate in X_cate:
            for lab in label:
                labelCounts[cate][lab] = 0
                    
        # Number of samples in a category        
        n_sample_cate = collections.defaultdict(int)
        
        # stores the counts of each label corresponding to each category of a feature
        for ind in range(m):
            labelCounts[X_cond[ind]][y_label[ind]] += 1
            n_sample_cate[X_cond[ind]] += 1
            
        # calculate the new entropy
        condShannonEntropy = 0.0
        # calculate the denomonator
        H = 0.0
        for cate_key,label_dict in labelCounts.items():

        
            for label_key in label_dict:
                # calculate occurrence
                prob = float(labelCounts[cate_key][label_key]/n_sample_cate[cate_key])

                if cate_key not in n_sample_cate:
                    n_sample_cate[cate_key] = 0
                
                if prob == 0:
                    continue
                # calculate entropy
                condShannonEntropy -= prob * np.log2(prob)

            y_prob = float(n_sample_cate[cate_key]/m)
            if y_prob == 0:
                continue
            # calculate denomonator H
            H -=  y_prob * np.log2(y_prob)    
        
        return condShannonEntropy, H

    # Select Feature from Max Information Gain
    
    def _feature_Select_Info_Gain(self, X, y_label):
        """
        Parameters
        ------------------
        X: {array-like}, shape = [n_samples, n_features]
        
        y_label: {array-like}, shape = [n_samples, 1]
        
        """
        m,n = X.shape
        #informationGain = []
        shannonEntropy = self.calcShannonEntropy(y_label)
        
        max_gain = 0.0
        best_feature = 0
        best_split = None
        
        for feature in range(n):
            
            X_cate = set(X[:,feature])
            
            for split_point in list(X_cate)[1:]:   
                
                # condShannonEntropy, H 
                
                condShannonEntrop, H = self.conditionalEnropy(X,y_label,feature,split_point)
                
                informationGain = shannonEntropy - condShannonEntrop

                
                if informationGain > max_gain:
                    max_gain = informationGain 
                    best_split = split_point
                    best_feature = feature
        
        if self.verbose == 1:
            print('>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
            print('\n')
            print("The best feature is {}, and it's best split point is {}".format(best_feature,best_split))
            print('\n')
        
        return best_feature, best_split
    
    # Select Feature from Max Information Gain Ratio
    
    def _feature_Select_Info_Gain_Ratio(self, X, y_label):
        """
        
        """
        m,n = X.shape
        GainRatio = []
        shannonEntropy = self.calcShannonEntropy(y_label)
        
        max_gain = 0.0
        best_feature = 0
        best_split = None
        
        for feature in range(n):
            X_cate = set(X[:,feature])
            
            for split_point in list(X_cate)[1:]: 
            
                condShannonEntropy, H = self.conditionalEnropy(X,y_label,feature,split_point)
#                 GainRatio.append((shannonEntropy - condShannonEntropy)/H)
                gainRatio = (shannonEntropy - condShannonEntropy)/H


                if gainRatio > max_gain:
                    max_gain = gainRatio 
                    best_split = split_point
                    best_feature = feature

#         feature = GainRatio.index(max(GainRatio))
        if self.verbose == 1:
            print('>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
            print('\n')
            print("The best feature is {}, and it's best split point is {}".format(best_feature,best_split))
            print('\n')
        return best_feature,best_split
    
    
    # Select Feature from Min Gini
    def gini_Impurity(self, y_labels):
        """
        Parameters
        ------------------
        y_labels: {array-like}, shape = [n_samples, 1]
            Dateset matrix, where 'n_samples' is the number of samples 
            and one column of labels
        
        Return
        ------------------
        giniImpurity: float
        
        """
        m = len(y_labels)
        
        labelCounts = collections.defaultdict(int)
        
        # the number of unique elements and their occurrence
        for label in y_labels:
            if label not in labelCounts:
                labelCounts[label] = 0
            labelCounts[label] += 1
            
        # calculate gini index
        gini = 1.0
        for key in labelCounts:
            # calculate occurrence
            prob = float(labelCounts[key]/m)
            # calculate entropy
            gini -= prob ** 2 
        
        return gini
    
    
    def total_Gini_Impurity(self, X, y_label, feature):
        """
        Parameters
        ------------------
        X: {array-like} discrete features
        preprocessed dataset, could not deal with continuous features, 
        and categorical feature should be better as binary form
        
        y_label: {array-like}
        
        feature: string
        
        Return
        ------------------
        split_point: it could be several different types: float, int, tring
        
        """
        m,n = X.shape
        
        X_cond = X[:,feature]
        
        # Create a nested dictionary, 
        # {"X_cate":{"label":count}}
        
        # Find unique category in X_cond
        X_cate = sorted(set(X_cond))
        # Find unique label in y_label
        label = set(y_label)
        
        split_point = collections.defaultdict(float)
        
        for i in list(X_cate)[1:]:
            """
            X_cond[i] == split point
            if yes, then left child, if no, then right child.
            """

            left_child = y_label[np.where(X_cond <= i)[0]]
            left_gini = self.gini_Impurity(left_child)

            right_child = y_label[np.where(X_cond > i)[0]]
            right_gini = self.gini_Impurity(right_child)
            
            totalGiniScore = float(len(left_child) * left_gini) + float(len(right_child)*right_gini)

            split_point[i] = totalGiniScore
            
        split =  min(split_point, key= split_point.get)
     
        return split_point[split], split
        
        
    def _feature_Select_Gini(self,X,y_labels):
        m,n = X.shape
        GiniScore = []
        Splits = []
        
        for feature in range(n):
            gini, split_point = self.total_Gini_Impurity(X,y_labels,feature)
            GiniScore.append(gini)
            Splits.append(split_point)
        
        best_feature = GiniScore.index(min(GiniScore))
        best_split = Splits[feature]
        if self.verbose == 1:
            print('>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
            print('\n')
            print("The best feature is {}, and it's best split point is {}".format(best_feature,best_split))
            print('\n')   
        return best_feature, best_split
        
    def featureSelect(self, X, y_label):
        
        best_feature, best_split = self.feature_split(X, y_label)
        
        return best_feature, best_split
    
    
    """
    Step2:
    BUILD A TREE
    
    """
    
    def buildTree(x_data,y_labels):
        pass
    

In [582]:
def createDataSet():
    dataSet = np.array([[2, 2],
            [2, 1],
            [0, 2],
            [0, 1],
            [0, 1]])
    
    labels = np.array(['yes',
              'yes',
              'no',
              'no',
              'no'])
    return dataSet, labels

In [583]:
dataSet ,labels =  createDataSet()
dt = DecisionTreeClassifier(criterion='Gini')
dt.featureSelect(dataSet,labels)

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


The best feature is 0, and it's best split point is 2




(0, 2)